In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from tools.metrics import diversity, uniqueness, traversal_index
import tlc

In [4]:
table = tlc.Table.from_url("C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/chessvision-classification/datasets/chesspieces-val/tables/val-cleaned-filtered")

In [ ]:
table.set_row_cache_url("./row_cache.parquet")
table.get_rows_as_binary()

In [6]:
import json

init_params = json.loads((table.url / "object.3lc.json").read())
init_params["row_cache_url"] = tlc.Url(init_params["row_cache_url"]).to_absolute(table.url)
from tlc.core.objects.tables.from_url.table_from_row_cache import TableFromRowCache

table_from_row_cache = TableFromRowCache(init_params)


In [8]:
label_column = table_from_row_cache.get_column("label")

In [9]:
metrics_table = tlc.Table.from_url("C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/chessvision-classification/runs/sienna-cheeseburger/reduced_0001")

In [ ]:
metrics_table.get_foreign_table_url().to_absolute(metrics_table.url) == table.url

In [ ]:
metrics_table.columns

In [12]:
embeddings_column = metrics_table.get_column("embeddings_pacmap")

In [13]:
import numpy as np

label_column = np.array(label_column.to_numpy().tolist(), dtype=np.float32)
embeddings_column = np.array(embeddings_column.to_numpy().tolist(), dtype=np.float32)

In [14]:
diversity = diversity(label_column, embeddings_column)
uniqueness = uniqueness(label_column, embeddings_column)
traversal = traversal_index(embeddings_column)

In [15]:
# run = tlc.Run.from_url("C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/chessvision-classification/runs/sienna-cheeseburger")

# run.add_metrics_data(metrics={"image_diversity": diversity}, input_table_url=table.url, stream_name="diversity")
# run.add_metrics_data(metrics={"image_uniqueness": uniqueness}, input_table_url=table.url, stream_name="uniqueness")
# run.add_metrics_data(metrics={"traversal_index": [int(t) for t in traversal]}, input_table_url=table.url, stream_name="traversal")

In [ ]:
from tools import add_columns_to_table

added_columns_table = add_columns_to_table(
    table,
    columns={
        "image_diversity": diversity,
        "image_uniqueness": uniqueness,
        "traversal_index": traversal,
    },
)